# [1단계] 라이브러리 설치

## 1-1. 라이브러리 설치

Colab 환경에서 필요한 라이브러리를 설치합니다. Hugging Face Hub에 접근하기 위해 `huggingface_hub`도 같이 설치합니다.

In [ ]:
!pip install -q datasets transformers accelerate torch ragas huggingface_hub

## 1-2. Hugging Face 로그인

HF_TOKEN을 이용해서 허깅페이스 허브에 로그인합니다.


In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)

# [2단계] 데이터셋 불러오기 및 전처리


## 2-1. Hugging Face 데이터셋 불러오기

팀원이 올려둔 KB 금융 데이터셋을 불러옵니다. 여기서는 validation split을 사용합니다.


In [ ]:
from datasets import load_dataset

dataset_name = "sssssungjae/finance-kb-mixed-dataset-final"
hf_ds = load_dataset(dataset_name, split="validation")

print(hf_ds)

## 2-2. Qwen 형식 파싱 (question / ground_truth 추출)

데이터셋은 Qwen 학습 포맷(`<|im_start|>user ... <|im_end|>`, `<|im_start|>assistant ... <|im_end|>`)으로 되어 있습니다.  
이를 ragas 평가에 맞게 `question`과 `ground_truth`로 분리합니다.


In [ ]:
def parse_qwen_text(row):
    text = row["text"]
    try:
        user_part = text.split("<|im_start|>user")[1].split("<|im_end|>")[0].strip()
        assistant_part = text.split("<|im_start|>assistant")[1].split("<|im_end|>")[0].strip()
    except:
        user_part, assistant_part = None, None
    return {"question": user_part, "ground_truth": assistant_part}

parsed_dataset = hf_ds.map(parse_qwen_text)

# contexts는 없으므로 ragas 형식에 맞춰 더미 값 추가
parsed_dataset = parsed_dataset.add_column("contexts", [[""]] * len(parsed_dataset))

parsed_dataset = parsed_dataset.remove_columns(["text"])
print(parsed_dataset[0])


# [3단계] 평가 대상 모델 로딩


## 3-1. 파인튜닝된 금융 SLM 로딩

Hugging Face Hub에 업로드된 파인튜닝 모델을 불러옵니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

my_model_id = "sssssungjae/qwen3-8b-finance-full"
model = AutoModelForCausalLM.from_pretrained(
    my_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

print("모델 로딩 완료 ✅")

# [4단계] 모델 답변 생성



## 4-1. 질문에 대해 모델이 새 답변 생성

`question`을 입력으로 넣고, 파인튜닝 모델이 답변을 생성합니다.

In [ ]:
from tqdm import tqdm

questions = parsed_dataset["question"]
contexts = parsed_dataset["contexts"]
ground_truths = parsed_dataset["ground_truth"]

generated_answers = []

for question, context in tqdm(zip(questions, contexts), total=len(questions)):
    prompt = f"""Use the following CONTEXT to answer the QUESTION.
    CONTEXT:
    {"\n".join(context if isinstance(context, list) else [context])}
    QUESTION:
    {question}
    """
    chat_prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=512)
    answer = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    generated_answers.append(answer)

print("답변 생성 완료 ✅")


# [5단계] Ragas 평가


## 5-1. ragas용 데이터셋 구성

질문(question), 답변(answer), 문맥(contexts), 정답(ground_truth)으로 최종 평가 데이터셋을 구성합니다.

In [ ]:
from datasets import Dataset

final_eval_data = {
    "question": questions,
    "answer": generated_answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}
final_eval_dataset = Dataset.from_dict(final_eval_data)


## 5-2. ragas 평가 실행

faithfulness, answer_relevancy, context_precision, context_recall 4가지 지표를 평가합니다.


In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

result = evaluate(
    dataset=final_eval_dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

# 상세 결과
result_df = result.to_pandas()
display(result_df.head())

# 평균 점수
summary = {k: round(v, 3) for k, v in result.items()}
print("\n평균 점수:", summary)